In [124]:
import numpy as np
from scipy.sparse import random as random_sparse
import random

Create random Sparse Nonegative Matrix uniformly distributed between 0 and 1

In [252]:

p = 4 #number of states
states = [i for i in range(p)]
P = random_sparse(p, p, density=0.1).toarray()


Make sure the matrix is ergodic

In [253]:
from scipy.stats import *

cycle_states = [i for i in range(p)]
random.shuffle(cycle_states)
# print(cycle_states)
walk_values = uniform.rvs(size=p+1)

for i, v in enumerate(walk_values):
    P[i % p, ((i + 1) % p)] = v
    

Turn matrix into a markovian matrix (Normalize rows by row-wise sum so it adds up to 1)

In [137]:
row_sum =  np.sum(P, axis=1).reshape(-1, 1)
M = np.divide(P, row_sum)
# print(M)
# print(np.sum(M, axis=1))


Verify the Matrix is Markovian (Row wise sum equals 1 or sends p-unity to the p-unity)

In [138]:
print(row_sum.shape)
print(M.shape)
Sp = M @ np.array([1]*p)
# print(Sp)

(5, 1)
(5, 5)


## Generate the Matrix as in the Paper

This method ensures low rank with a non-zero entry matrix

In [238]:
r = 3 #Desired rank of the transition matrix
np.random.seed(0)
U = np.random.standard_normal(size=(p, r))
V = np.random.standard_normal(size=(p, r))

def normalizeMatrix(H, axis=1):
    axis_sum =  np.sum(H, axis=axis)
    if axis:
        return np.divide(H, axis_sum.reshape(-1, 1))
    else:
        return np.divide(H, axis_sum.reshape(1, -1))


U1 = normalizeMatrix(U * U)
V1 = normalizeMatrix(V * V, axis = 0)

# print(np.sum(U1, axis=1))
# print(np.sum(V1, axis=1))

M = U1 @ V1.transpose()

# print(M)
print(np.linalg.matrix_rank(M))
# print(np.sum(M, axis=1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
[0.04252461 0.01088628 0.0393806  0.01249095 0.01897883 0.00095953
 0.00552757 0.01544101 0.01431497 0.06386591 0.02487003 0.08348189
 0.03702153 0.03981056 0.02040235 0.00226633 0.01049452 0.04258562
 0.04201971 0.00838724 0.01381519 0.00140595 0.00255496 0.02347514
 0.0075582  0.03189445 0.03761585 0.05975651 0.03569134 0.04685995
 0.00491945 0.03246953 0.04171884 0.02341894 0.03946592 0.0074597
 0.02816716 0.00774377 0.03526563 0.05246937 0.05724746 0.00039977
 0.0820525  0.02361602 0.01792931 0.0072641  0.00797169 0.01365582
 0.0098278  0.01496834 0.01941743 0.04429563 0.04773835 0.01248103
 0.00482794 0.07226298 0.09426702 0.00880092 0.05698375 0.02207144
 0.02430258 0.08201249 0.

## Generate matrix by hand

In [278]:
p = 4
M = np.array([[0.2, 0.25, 0.25, 0.3], [0.2, 0.25, 0.3, 0.25], [0, 0, 0.8, 0.2], [0, 0, 0.4, 0.6]])
print(M)
print(svd(M)[1])
print(M @ np.array([1]*p))
print(np.linalg.matrix_rank(M))

[[0.2  0.25 0.25 0.3 ]
 [0.2  0.25 0.3  0.25]
 [0.   0.   0.8  0.2 ]
 [0.   0.   0.4  0.6 ]]
[1.17839549e+00 4.51105766e-01 3.43347716e-01 7.08340353e-17]
[1. 1. 1. 1.]
3


## Algorithm


Simulate Random Walk of Size *n*

In [262]:
current_state = np.random.choice(states)
path = [current_state]
# b = np.hstack(M[0])
# print(b.shape)
# n = round(30 *p * np.log(p))
n = 10
for i in range(n):
    # print(i, end='')
    current_state = np.random.choice(states, p = M[current_state])
    path.append(current_state)


In [263]:
print(f"Number of samples: {n}")

Number of samples: 10


Display True Transition Matrix


In [260]:
print(M)

[[0.2  0.25 0.25 0.3 ]
 [0.2  0.25 0.3  0.25]
 [0.   0.   0.8  0.2 ]
 [0.   0.   0.4  0.6 ]]


Counting State transition based on random walk data

In [265]:
N = np.zeros((p, p))
for k in range(n):
    i, j = path[k], path[k + 1]
    N[i, j] += 1
print(N)
Ni = np.sum(N, axis=1).flatten()
print(Ni)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 5. 2.]
 [0. 0. 2. 1.]]
[0. 0. 7. 3.]


## Algorithm Initial Point

In [279]:
from scipy.linalg import block_diag
from scipy.stats import ortho_group

#Parameters
iter_number = 1000
penalty = 0.5
step_len = 0.5
c = 3 #Rank constraint??

#Constants
b = np.ones(shape=(p, 1))
A = block_diag(*[b.transpose()]*p)
index = np.copy(N)
index[index != 0] = 1 #All nonzero entries of our transition state counts are convert into 1
not_index = - (index - 1)

def AstarFunc(y):
    # print("y shape", y.shape)
    return (A.transpose() @ y).reshape((p, p))
def AFunc(M):
    # print(y.flatten().transpose().shape)
    return M @ b

#Don't know any heuristics for a 'good' initial point
S = ortho_group.rvs(p) @ np.diag(uniform.rvs(size=p) * c) @ ortho_group.rvs(p).T # random matrix with 0 < spectral norm < c. 
X = N
X = np.divide(X, np.sum(X, axis=1).reshape(-1, 1))
y = np.random.rand(p, 1)
Xi = -(S + AstarFunc(y))

# print(A @ A.transpose())
# print(np.linalg.norm(S, ord=2))
# print(np.linalg.norm(Xi + S + AstarFunc(y)))


C:\Users\diego\AppData\Local\Temp\ipykernel_31080\284028935.py:27: RuntimeWarning: invalid value encountered in true_divide
  X = np.divide(X, np.sum(X, axis=1).reshape(-1, 1))


Initial point by hand

In [298]:
N = np.array([[1, 0, 0, 0], [1, 1, 1, 1], [0, 2, 4, 2], [0, 1, 2, 1]])
n = 17
print(N)

[[1 0 0 0]
 [1 1 1 1]
 [0 2 4 2]
 [0 1 2 1]]


In [307]:
## First matrix S
U = np.array([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]) #90deg rotation on the x-y plane
D = np.diag([2, 2, 1, 0])
print( U @ U.transpose())
print(D)
X = N
X = np.divide(X, np.sum(X, axis=1).reshape(-1, 1))
S = U @ D
print(S)
print(svd(S)[1])
print(X)

penalty = 0.5
step_len = 0.5
c = 2 #Rank constraint??

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[[2 0 0 0]
 [0 2 0 0]
 [0 0 1 0]
 [0 0 0 0]]
[[ 0 -2  0  0]
 [ 2  0  0  0]
 [ 0  0  1  0]
 [ 0  0  0  0]]
[2. 2. 1. 0.]
[[1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.25 0.5  0.25]
 [0.   0.25 0.5  0.25]]


In [308]:
# Y and Xi
y = np.array([[0.5, 0.5, 0, 1]]).transpose()
Xi = -(S + AstarFunc(y))
print(y)
print(AstarFunc(y))
print(Xi + S + X/penalty)
print(computeY(Xi, S, X))

[[0.5]
 [0.5]
 [0. ]
 [1. ]]
[[0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5]
 [0.  0.  0.  0. ]
 [1.  1.  1.  1. ]]
[[ 1.5 -0.5 -0.5 -0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.5  1.   0.5]
 [-1.  -0.5  0.  -0.5]]
[[0.5]
 [0.5]
 [0. ]
 [1. ]]


In [310]:
##S
W = -(Xi + AstarFunc(y) + (X / penalty))
print(W)
Uo, s, Vh = svd(W, full_matrices=False) 
print(s)
S2 = computeS(Xi, y, X)
print(svd(S2)[1])

[[-2.  -2.  -0.  -0. ]
 [ 1.5 -0.5 -0.5 -0.5]
 [-0.  -0.5 -0.  -0.5]
 [-0.  -0.5 -1.  -0.5]]
[2.97316067 1.86227807 0.74935615 0.36152645]
[2.         1.86227807 0.74935615 0.36152645]


In [321]:
## Xi
R = AstarFunc(y) + S + X/penalty
print(R)
index = np.copy(N)
index[index != 0] = 1 #All nonzero entries of our transition state counts are convert into 1
not_index = - (index - 1)
print((2.5 + np.sqrt(2.5**2 + 8/17))/(4))
print(penalty*(R[0,0] + np.sqrt(R[0,0]**2 + 4*N[0,0]/(n * penalty)))/(2))
print((((2.5 + np.sqrt(2.5**2 + 8/17))/(4))- 2.5/2)*2)
print((1.2731024338064798 - 2.5/2)*2)

Xi2 = computeXi(y, S, X)
print(Xi2)
print(not_index)

[[ 2.5 -1.5  0.5  0.5]
 [ 3.   1.   1.   1. ]
 [ 0.   0.5  2.   0.5]
 [ 1.   1.5  2.   1.5]]
1.2731024338064798
1.2731024338064798
0.04620486761295961
0.04620486761295961
[[0.04620487 1.5        0.         0.        ]
 [0.03871604 0.10633906 0.10633906 0.10633906]
 [0.         0.29570516 0.21267813 0.29570516]
 [0.         0.07471029 0.11143786 0.07471029]]
[[0 1 1 1]
 [0 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [292]:
print(X)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 5. 2.]
 [0. 0. 2. 1.]]


### Computation

In [301]:
from scipy.linalg import svd

def computeY(Xi, S, X):
    y = (1 / (penalty * p)) * (b - penalty*AFunc(Xi + S + (1/penalty)* X))
    return y

def computeXi(y, S, X):
    R = AstarFunc(y) + S + X * (1 / penalty)
    Z = penalty * (index * 0.5 * (R + np.sqrt(R ** 2 + 4 * N / (n * penalty))) + not_index * np.maximum(R, 0) )
    return (1 / penalty) * (Z - penalty * R)
    

def computeS(Xi, y, X):
    W = -(Xi + AstarFunc(y) + (X / penalty))
    U, s, Vh = svd(W, full_matrices=False) #Computes de partial SVD of W
    # print(s)
    return U @ np.diag(np.minimum(s, c)) @ Vh
    


In [37]:
print((A @ (Xi + S + (1/penalty)* X).flatten()).shape)
print((Xi + S + (1/penalty)* X).flatten().shape)
A.transpose().shape

(10,)
(100,)


(100, 10)

In [249]:
for _ in range(200):
    # print(svd(S,  full_matrices=False)[1])
    # print(Xi)
    yk1_2 = computeY(Xi, S, X)
    Xi = computeXi(y, S, X)
    yk1 = computeY(Xi, S, X)
    S = computeS(Xi, yk1, X)
    X = X + step_len * penalty * (Xi + AstarFunc(yk1) + S)
    print(np.linalg.norm((X-M), ord='fro'))
    # print(_, end='')

1.1456538622870451
1.0095333008446663
0.9967088240459673
1.022337233416906
1.0484462781681176
1.0684239850463688
1.0826592209641919
1.0925684415871841
1.099405843789086
1.1041197117298285
1.1073752814719584
1.1096347922295515
1.1112240128096187
1.112356493488765
1.1131699488131217
1.1137598623786684
1.1141904757056789
1.114505972474606
1.1147368103717443
1.1149046441449249
1.115026343269383
1.1151149700847764
1.1151801546127638
1.1152287654472568
1.1152656268451924
1.115295528584062
1.1153203471962316
1.1153413543548638
1.1153594501957316
1.1153752750626442
1.115389288269132
1.1154018229243896
1.1154131239788034
1.1154233746012212
1.1154327144666534
1.115441252441831
1.1154490742514531
1.1154561351608645
1.1154625496714685
1.1154684083305075
1.1154737820899594
1.1154787271045563
1.115483288530294
1.115487503366505
1.1154914025375215
1.1154950124013236
1.115498355837314
1.115501453029515
1.1155043220313807
1.1155069791749197
1.115509439369228
1.1155117163205968
1.1155138226970367
1.1155

In [217]:
print(svd(S)[1])

[3.00000000e+00 8.14961374e-01 4.90800463e-01 4.68871724e-01
 2.57288810e-01 2.23513791e-01 2.07098855e-01 7.83268302e-02
 5.94566252e-02 3.68026452e-04]


In [200]:
print(X)
# print(X @ b)
# print(np.linalg.norm((X-M), ord='fro'))

[[0.08760009 0.1038774  0.1052948  0.11034017 0.11997745 0.0995497
  0.09915211 0.1019819  0.08445389 0.08921931]
 [0.08754638 0.10384919 0.10498278 0.11035146 0.11993923 0.0996179
  0.0990096  0.10193428 0.08438239 0.08922957]
 [0.08730434 0.10369823 0.10461109 0.11000161 0.11963867 0.09921214
  0.09885904 0.10164318 0.08418075 0.08892483]
 [0.08721143 0.10325241 0.10439898 0.11005077 0.11927223 0.09911843
  0.09861684 0.10173027 0.08406562 0.08879422]
 [0.08725575 0.10341982 0.10449376 0.10994813 0.11948407 0.09922113
  0.09876769 0.10155025 0.08410966 0.08886215]
 [0.08720033 0.10327578 0.10456392 0.10988719 0.11947913 0.0991792
  0.09853319 0.10152419 0.08406542 0.08882599]
 [0.08774669 0.10393624 0.10507174 0.11075505 0.12015065 0.09980533
  0.09932638 0.10231174 0.08459885 0.08941195]
 [0.08788356 0.10420942 0.10526433 0.11076236 0.12042689 0.09994306
  0.09933446 0.10232141 0.08465336 0.08948565]
 [0.08654214 0.10247882 0.10362962 0.10900686 0.11917949 0.09834557
  0.09782527 0.

Divergence 

In [388]:
R = np.random.rand(p, p) * 2 - 1
R

array([[-0.05192858, -0.02106327,  0.7623498 ,  0.12689153, -0.20668074,
         0.38171573, -0.22684618,  0.05700147,  0.03250422,  0.68304834],
       [-0.9213023 ,  0.15364065, -0.21977149,  0.38887912,  0.57493247,
        -0.27710732, -0.23335893,  0.45009374,  0.27842198, -0.0835563 ],
       [-0.21251501, -0.50032127,  0.22295982, -0.08145476, -0.17847434,
        -0.00711511,  0.82355032,  0.84480824, -0.02223802,  0.10432343],
       [ 0.82448126, -0.1504518 , -0.96665194, -0.38616322, -0.31939372,
        -0.61070449,  0.10989224,  0.08799741, -0.43797771,  0.61648308],
       [ 0.4743206 , -0.95398916,  0.96938304,  0.48428192,  0.64037873,
         0.93023749,  0.50160421, -0.6809764 ,  0.80659214, -0.29842931],
       [ 0.70718733, -0.81918708, -0.56165384,  0.5036696 ,  0.0035726 ,
         0.91299703,  0.97030812, -0.08711916,  0.76505666, -0.20310429],
       [ 0.94777645, -0.92345866, -0.43484322, -0.99842342,  0.30478596,
         0.39805816, -0.15135518,  0.38266725

In [389]:
index = np.copy(N)
index[index != 0] = 1
print(index)
# print(M)
Z = -penalty * np.maximum(R, 0) * (index - 1) + index * 0.5 * penalty * (R + np.sqrt(R ** 2 + 4 * N/(n * penalty)))
(1 / penalty) * (Z - penalty * R)


[[0. 1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


array([[0.05192858, 0.39797313, 0.        , 0.        , 0.20668074,
        0.24091375, 0.22684618, 0.        , 0.        , 0.        ],
       [0.9213023 , 0.        , 0.51247099, 0.        , 0.        ,
        0.27710732, 0.23335893, 0.        , 0.        , 0.0835563 ],
       [0.21251501, 0.50032127, 0.        , 0.43016122, 0.17847434,
        0.00711511, 0.        , 0.        , 0.02223802, 0.        ],
       [0.        , 0.1504518 , 0.96665194, 0.38616322, 0.43447511,
        0.61070449, 0.        , 0.18389576, 0.53196828, 0.        ],
       [0.        , 0.95398916, 0.        , 0.        , 0.        ,
        0.05095823, 0.        , 0.6809764 , 0.        , 0.29842931],
       [0.        , 0.81918708, 0.56165384, 0.        , 0.        ,
        0.        , 0.17467506, 0.08711916, 0.        , 0.20310429],
       [0.0501061 , 0.92345866, 0.52930646, 0.99842342, 0.        ,
        0.        , 0.15135518, 0.17827232, 0.        , 0.78034249],
       [0.25391781, 0.        , 0.3058857

In [390]:
Z = penalty * (index * 0.5 * (R + np.sqrt(R ** 2 + 4 * N / (n * penalty))) + not_index * np.maximum(R, 0) )
print(np.maximum(R, 0))
# print(R)
(1 / penalty) * (Z - penalty * R)

[[0.         0.         0.7623498  0.12689153 0.         0.38171573
  0.         0.05700147 0.03250422 0.68304834]
 [0.         0.15364065 0.         0.38887912 0.57493247 0.
  0.         0.45009374 0.27842198 0.        ]
 [0.         0.         0.22295982 0.         0.         0.
  0.82355032 0.84480824 0.         0.10432343]
 [0.82448126 0.         0.         0.         0.         0.
  0.10989224 0.08799741 0.         0.61648308]
 [0.4743206  0.         0.96938304 0.48428192 0.64037873 0.93023749
  0.50160421 0.         0.80659214 0.        ]
 [0.70718733 0.         0.         0.5036696  0.0035726  0.91299703
  0.97030812 0.         0.76505666 0.        ]
 [0.94777645 0.         0.         0.         0.30478596 0.39805816
  0.         0.38266725 0.86265025 0.        ]
 [0.         0.41191779 0.         0.34527799 0.         0.
  0.         0.24050102 0.         0.        ]
 [0.         0.         0.         0.78494588 0.         0.45578988
  0.0215603  0.         0.         0.       

array([[0.05192858, 0.39797313, 0.        , 0.        , 0.20668074,
        0.24091375, 0.22684618, 0.        , 0.        , 0.        ],
       [0.9213023 , 0.        , 0.51247099, 0.        , 0.        ,
        0.27710732, 0.23335893, 0.        , 0.        , 0.0835563 ],
       [0.21251501, 0.50032127, 0.        , 0.43016122, 0.17847434,
        0.00711511, 0.        , 0.        , 0.02223802, 0.        ],
       [0.        , 0.1504518 , 0.96665194, 0.38616322, 0.43447511,
        0.61070449, 0.        , 0.18389576, 0.53196828, 0.        ],
       [0.        , 0.95398916, 0.        , 0.        , 0.        ,
        0.05095823, 0.        , 0.6809764 , 0.        , 0.29842931],
       [0.        , 0.81918708, 0.56165384, 0.        , 0.        ,
        0.        , 0.17467506, 0.08711916, 0.        , 0.20310429],
       [0.0501061 , 0.92345866, 0.52930646, 0.99842342, 0.        ,
        0.        , 0.15135518, 0.17827232, 0.        , 0.78034249],
       [0.25391781, 0.        , 0.3058857

In [391]:
W = -(Xi + AstarFunc(y) + (1 / penalty) * X)
U, s, Vh = svd(W, full_matrices=False)
# print(W)
# print(U @ np.diag(s) @ Vh)
print(s)
np.minimum(s, c)

[8.47349994e+01 1.15215110e-01 9.18088479e-02 6.24781823e-02
 3.13113826e-02 2.28155098e-02 1.96844921e-02 1.40834422e-02
 5.62155618e-03 2.89322336e-03]


array([4.00000000e+00, 1.15215110e-01, 9.18088479e-02, 6.24781823e-02,
       3.13113826e-02, 2.28155098e-02, 1.96844921e-02, 1.40834422e-02,
       5.62155618e-03, 2.89322336e-03])

In [392]:
from scipy.stats import ortho_group
U = ortho_group.rvs(3)
V = ortho_group.rvs(3)

s = np.diag(uniform.rvs(size=3) * 3)
H = U @ s @ V.T
a, b, c = svd(H)
print(s)

[[0.42715686 0.         0.        ]
 [0.         1.07438104 0.        ]
 [0.         0.         1.745115  ]]


In [405]:
print(M)
t = svd(M)
t[1]

[[0.         0.00521323 0.         0.         0.         0.
  0.         0.         0.99478677 0.        ]
 [0.         0.         0.34798682 0.         0.21439699 0.
  0.1633798  0.         0.         0.2742364 ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.53547528 0.
  0.         0.         0.46452472 0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.63917481 0.         0.36082519 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.        ]
 [0.04307161 0.37045709 0.         0.         0.39067867 0.
  0.         0.         0.19579264 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.53372806 0.         0.46627194]
 [1.         0.         0.         0.

array([1.24112731, 1.15941607, 1.00106923, 1.        , 1.        ,
       0.67969834, 0.64496328, 0.47639652, 0.29523912, 0.23088409])

Prueba manual


In [251]:
p = 4
P = np.array([[0.2, 0, 0, 0.8], [0, 0.25, 0.5, 0.25], [0, 0, 0.8, 0.2], [0, 0, 0.4, 0.6]])
print(P)
print(svd(P)[1])
print(P @ np.array([1]*p))

[[0.2  0.   0.   0.8 ]
 [0.   0.25 0.5  0.25]
 [0.   0.   0.8  0.2 ]
 [0.   0.   0.4  0.6 ]]
[1.27338155 0.7599172  0.21661268 0.09541602]
[1. 1. 1. 1.]
